In [61]:
import transformers
import torch
import pandas as pd
import huggingface_hub
import pickle

from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, BartForSequenceClassification, PreTrainedTokenizerFast
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from transformers import get_scheduler

In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file='tokenizer.json')

In [7]:
class ConversationDataset(Dataset):
    def __init__(self):
        self.data = pd.read_csv('conv_data.csv', encoding='utf-8', index_col=0)
        self.tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-base-v2')
        self.emo_code = pd.read_csv('emotion_code.csv', encoding='utf-8', index_col=0)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        train_data = tokenizer(f"<s>{self.data.iloc[index]['conversation']}</s>", return_tensors='pt', truncation=True, padding="max_length", max_length=300)
        train_data['label'] = torch.tensor(self.emo_code[self.emo_code['code'] == self.data.iloc[index]['emotion']].index[0])
        return train_data.to("cuda")

In [8]:
dataset = ConversationDataset()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [9]:
train, val = train_test_split(dataset, test_size=0.2, random_state=42)
val, test = train_test_split(val, test_size=0.5, random_state=42)

In [10]:
train_dataset = DataLoader(train, batch_size=4, shuffle=True)
val_dataset = DataLoader(val, batch_size=4)
test_dataset = DataLoader(test, batch_size=4)

In [11]:
class LyricEmotionClassifier:
    def __init__(self):
        self.model = BartForSequenceClassification.from_pretrained('gogamza/kobart-base-v2', num_labels=60).to("cuda")
        self.optimizer = AdamW(self.model.parameters(), lr=5e-5)
        self.lr_scheduler = get_scheduler(
            name="linear",
            optimizer=self.optimizer,
            num_warmup_steps=0,
            num_training_steps=len(train_dataset),
        )
        
    def model_train(self):
        self.model.train()
        self.total_train_loss = 0
        
        for batch in tqdm(train_dataset):
            input_ids = batch['input_ids'].to(device).squeeze()
            attention_mask = batch['attention_mask'].to(device).squeeze()
            labels = batch['label'].to(device)
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            self.total_train_loss += loss.item()

            # 역전파
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.lr_scheduler.step()

        avg_train_loss = self.total_train_loss / len(train_dataset)
        print(f"training loss: {avg_train_loss}")

In [12]:
bart1 = LyricEmotionClassifier()
bart1.model.load_state_dict(torch.load('bart_lyric.pt'))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at gogamza/kobart-base-v2 and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
with open('emotion_code.pkl', 'rb') as file:
    emo_list = pickle.load(file)

In [65]:
emo_list

[{'idx': 0, 'code': 'E10', 'emotion': '분노'},
 {'idx': 1, 'code': 'E11', 'emotion': '툴툴대는'},
 {'idx': 2, 'code': 'E12', 'emotion': '좌절한'},
 {'idx': 3, 'code': 'E13', 'emotion': '짜증내는'},
 {'idx': 4, 'code': 'E14', 'emotion': '방어적인'},
 {'idx': 5, 'code': 'E15', 'emotion': '악의적인'},
 {'idx': 6, 'code': 'E16', 'emotion': '안달하는'},
 {'idx': 7, 'code': 'E17', 'emotion': '구역질나는'},
 {'idx': 8, 'code': 'E18', 'emotion': '노여워하는'},
 {'idx': 9, 'code': 'E19', 'emotion': '성가신'},
 {'idx': 10, 'code': 'E20', 'emotion': '슬픔'},
 {'idx': 11, 'code': 'E21', 'emotion': '실망한'},
 {'idx': 12, 'code': 'E22', 'emotion': '비통한'},
 {'idx': 13, 'code': 'E23', 'emotion': '후회되는'},
 {'idx': 14, 'code': 'E24', 'emotion': '우울한'},
 {'idx': 15, 'code': 'E25', 'emotion': '마비된'},
 {'idx': 16, 'code': 'E26', 'emotion': '염세적인'},
 {'idx': 17, 'code': 'E27', 'emotion': '눈물이나는'},
 {'idx': 18, 'code': 'E28', 'emotion': '낙담한'},
 {'idx': 19, 'code': 'E29', 'emotion': '환멸을느끼는'},
 {'idx': 20, 'code': 'E30', 'emotion': '불안'},
 {'idx': 2

In [66]:
emo_list1 = [{row['idx']: {'code': row['code'], 'emotion': row['emotion']} } for row in emo_list]

In [67]:
emo_list1

[{0: {'code': 'E10', 'emotion': '분노'}},
 {1: {'code': 'E11', 'emotion': '툴툴대는'}},
 {2: {'code': 'E12', 'emotion': '좌절한'}},
 {3: {'code': 'E13', 'emotion': '짜증내는'}},
 {4: {'code': 'E14', 'emotion': '방어적인'}},
 {5: {'code': 'E15', 'emotion': '악의적인'}},
 {6: {'code': 'E16', 'emotion': '안달하는'}},
 {7: {'code': 'E17', 'emotion': '구역질나는'}},
 {8: {'code': 'E18', 'emotion': '노여워하는'}},
 {9: {'code': 'E19', 'emotion': '성가신'}},
 {10: {'code': 'E20', 'emotion': '슬픔'}},
 {11: {'code': 'E21', 'emotion': '실망한'}},
 {12: {'code': 'E22', 'emotion': '비통한'}},
 {13: {'code': 'E23', 'emotion': '후회되는'}},
 {14: {'code': 'E24', 'emotion': '우울한'}},
 {15: {'code': 'E25', 'emotion': '마비된'}},
 {16: {'code': 'E26', 'emotion': '염세적인'}},
 {17: {'code': 'E27', 'emotion': '눈물이나는'}},
 {18: {'code': 'E28', 'emotion': '낙담한'}},
 {19: {'code': 'E29', 'emotion': '환멸을느끼는'}},
 {20: {'code': 'E30', 'emotion': '불안'}},
 {21: {'code': 'E31', 'emotion': '두려운'}},
 {22: {'code': 'E32', 'emotion': '스트레스받는'}},
 {23: {'code': 'E33', 'emoti

In [98]:
def eval_emo(model, lyrics):
    model.eval()
    inputs = tokenizer(lyrics, return_tensors="pt", padding=True, truncation=True, max_length=500)
    print(inputs)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        prob = torch.nn.functional.softmax(outputs[0], dim=1).squeeze().detach().cpu().numpy()
        label = torch.argsort(outputs[0], descending=True).squeeze().detach().cpu().numpy()
        print(sorted(prob, reverse=True)[:5])
        for code in label[:5]:
            print(emo_list[code]['emotion'], end=' ')

In [111]:
input_text = """
내 어린 시절 우연히 들었던 믿지 못할 한마디
이 세상을 다 준다는 매혹적인 얘기 내게 꿈을 심어주었어
말도 안돼 고개 저어도 내 안에 나 나를 보고 속삭여
세상은 꿈꾸는 자의 것이라고 용기를 내 넌 할 수 있어
쉼 없이 흘러가는 시간 이대로 보낼 수는 없잖아
함께 도전하는 거야 너와 나 두 손을 잡고 우리들 모두의 꿈을 모아서
외로움과 두려움이 우릴 힘들게 하여도 결코 피하지 않아
끝없이 펼쳐진 드넓은 바다에 희망이 우리를 부르니까
거센 바람 높은 파도가 우리 앞길 막아서도 결코 두렵지 않아
끝없이 펼쳐진 수많은 시련들 밝은 내일 위한 거야
말도 안돼 고개 저어도 내 안에 나 나를 보고 속삭여
세상은 꿈꾸는 자의 것이라고 용기를 내 넌 할 수 있어
쉼 없이 흘러가는 시간 이대로 보낼 수는 없잖아
함께 도전하는 거야 너와 나 두 손을 잡고 우리들 모두의 꿈을 모아서
외로움과 두려움이 우릴 힘들게 하여도 결코 피하지 않아
끝없이 펼쳐진 드넓은 바다에 희망이 우리를 부르니까
거센 바람 높은 파도가 우리 앞길 막아서도 결코 두렵지 않아
끝없이 펼쳐진 수많은 시련들 밝은 내일 위한 거야
원피스 접기
"""

In [112]:
eval_emo(bart1.model, f'<s>{input_text}</s>')

{'input_ids': tensor([[    0, 14095,  9517, 15079, 15908, 25177, 14172, 15912, 15325, 12332,
         17796, 19658,   230, 12034, 18105, 14056, 14450, 14090, 14174, 13700,
         14134, 15734, 18750, 18557, 25751, 12258, 11779, 11763,   230, 10504,
          9866, 27812, 14068,  9006, 14209, 16291, 14067, 16196, 14054, 16389,
         14581, 14299, 11208, 11802,   230, 11285, 15978, 27814, 24373, 15751,
         28739, 14067, 28007, 14207, 14032, 14389,   230, 11423, 14807, 17897,
         14542, 14489, 14025, 14320, 24819, 16228, 14080, 16223,   230, 27025,
         16523, 14049, 19454, 14567, 11863, 14054, 14196, 16512, 18398, 14199,
          9993, 14422, 12024, 18557, 17453, 11264,   230, 11890, 10338, 24678,
         18018, 15608, 14093, 10482, 15994,  9049, 16371,  9866, 17704, 14250,
         14242, 15296,   230,  9480, 14997, 16566, 12335, 14340,  9553, 12005,
         17017, 11786, 15724, 12034, 21539, 18062, 14495,   230,  9031, 11288,
         15394, 14904, 14240, 14973, 1